In [1]:
import pysam
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

In [2]:
FOLDER = '/Volumes/Seagate Expansion Drive/aci_storage/STAR.hyp/'

In [3]:
gene_coord = pd.read_csv('Zea_mays.AGPv4.32.gene.bed', sep='\t',header=None)
gene_coord.head()

,0,1,2,3,4
0,1,44289,49837,+,Zm00001d027230
1,1,50877,55716,-,Zm00001d027231
2,1,92299,95134,-,Zm00001d027232
3,1,111655,118312,-,Zm00001d027233
4,1,118683,119739,-,Zm00001d027234


In [4]:
THES = 0
def extract_kmer(seq, k=25):
    kmer = []
    for i in range(len(seq)-k+1):
        kmer.append(seq[i:i+k])
    return kmer

def query_bam(path, CHR, START, END, ITER=10000, THRES=255, k=25):

    df = pd.DataFrame(columns=['chr','start','end','seq', 'kmer'])
    bam = pysam.AlignmentFile(path, 'rb')
    j=0
    for i, read in tqdm_notebook(enumerate(bam.fetch(CHR, START, END))):
        if (read.flag) > THRES and (read.query_length>=k):
            kmer = extract_kmer(read.seq, k=k)
            df.loc[j,'chr'] = read.reference_name
            df.loc[j,'start'] = read.reference_start
            df.loc[j,'end'] = read.reference_end
            df.loc[j,'seq'] = read.seq
            df.loc[j,'kmer'] = kmer
            j += 1
        
    return df
        

In [5]:
UU1 = FOLDER + 'UU51/Aligned.sortedByCoord.out.bam'
UU2 = FOLDER + 'UU52/Aligned.sortedByCoord.out.bam'
UU3 = FOLDER + 'UU59/Aligned.sortedByCoord.out.bam'
UU4 = FOLDER + 'UU61/Aligned.sortedByCoord.out.bam'
UU5 = FOLDER + 'UU65/Aligned.sortedByCoord.out.bam'

WW1 = FOLDER + 'WW49/Aligned.sortedByCoord.out.bam'
WW2 = FOLDER + 'WW54/Aligned.sortedByCoord.out.bam'
WW3 = FOLDER + 'WW58/Aligned.sortedByCoord.out.bam'
WW4 = FOLDER + 'WW62/Aligned.sortedByCoord.out.bam'
WW5 = FOLDER + 'WW63/Aligned.sortedByCoord.out.bam'


df_uu1 = query_bam(UU1, "B73V4_ctg150", 0, 1563274)
df_uu2 = query_bam(UU2, "B73V4_ctg150", 0, 1563274)
df_uu3 = query_bam(UU3, "B73V4_ctg150", 0, 1563274)
df_uu4 = query_bam(UU4, "B73V4_ctg150", 0, 1563274)
df_uu5 = query_bam(UU5, "B73V4_ctg150", 0, 1563274)

df_ww1 = query_bam(WW1, "B73V4_ctg150", 0, 1563274)
df_ww2 = query_bam(WW2, "B73V4_ctg150", 0, 1563274)
df_ww3 = query_bam(WW3, "B73V4_ctg150", 0, 1563274)
df_ww4 = query_bam(WW4, "B73V4_ctg150", 0, 1563274)
df_ww5 = query_bam(WW5, "B73V4_ctg150", 0, 1563274)

In [6]:
kmer_uu1 = np.unique(sum(df_uu1['kmer'].values,[]), return_counts=True)
kmer_uu2 = np.unique(sum(df_uu2['kmer'].values,[]), return_counts=True)
kmer_uu3 = np.unique(sum(df_uu3['kmer'].values,[]), return_counts=True)
kmer_uu4 = np.unique(sum(df_uu4['kmer'].values,[]), return_counts=True)
kmer_uu5 = np.unique(sum(df_uu5['kmer'].values,[]), return_counts=True)

kmer_ww1 = np.unique(sum(df_ww1['kmer'].values,[]), return_counts=True)
kmer_ww2 = np.unique(sum(df_ww2['kmer'].values,[]), return_counts=True)
kmer_ww3 = np.unique(sum(df_ww3['kmer'].values,[]), return_counts=True)
kmer_ww4 = np.unique(sum(df_ww4['kmer'].values,[]), return_counts=True)
kmer_ww5 = np.unique(sum(df_ww5['kmer'].values,[]), return_counts=True)

### We looked at k-mers that have >=3 reads in each UU library and <=1 reads in each WW library

In [7]:
kmer_UU = ((set(kmer_uu1[0][kmer_uu1[1]>=3]) & set(kmer_uu2[0][kmer_uu2[1]>=3])) & 
           (set(kmer_uu3[0][kmer_uu3[1]>=3]) & set(kmer_uu4[0][kmer_uu4[1]>=3])) 
           & set(kmer_uu5[0][kmer_uu5[1]>=3]))


In [8]:
unique_UU = kmer_UU - set(kmer_ww1[0][kmer_ww1[1]>1]) - set(kmer_ww2[0][kmer_ww2[1]>1]) - set(kmer_ww3[0][kmer_ww3[1]>1]) - set(kmer_ww4[0][kmer_ww4[1]>1]) - set(kmer_ww5[0][kmer_ww5[1]>1])


In [9]:
print(len(unique_UU))
print(unique_UU)

53
{'ACACCGGCGCCGCCGCCTTCCTCGC', 'CCTTCCTCGCCGACGATGACCTCGG', 'TTCCTCGCCGACGATGACCTCGGCG', 'GGTTCTGTTGGTGCACTAATCCTGC', 'CGCCTTCCTCGCCGACGATGACCTC', 'CGCCGCCTTCCTCGCCGACGATGAC', 'AACACCGGCGCCGCCGCCTTCCTCG', 'GCGTTTTCAGGGTTCTGTTGGTGCA', 'TTTCTATGGCGTTTTCAGGGTTCTG', 'CGCCGCCGCCTTCCTCGCCGACGAT', 'TCAGGGTTCTGTTGGTGCACTAATC', 'GCCGCCGCCTTCCTCGCCGACGATG', 'TTTTTCTATGGCGTTTTCAGGGTTC', 'TGGCGTTTTCAGGGTTCTGTTGGTG', 'GGGTTCTGTTGGTGCACTAATCCTG', 'GCGCCGCCGCCTTCCTCGCCGACGA', 'TTCTATGGCGTTTTCAGGGTTCTGT', 'GCCGACGATGACCTCGGCGGGCTGT', 'CACCGGCGCCGCCGCCTTCCTCGCC', 'CAACTTTTTCTATGGCGTTTTCAGG', 'GCCGCCTTCCTCGCCGACGATGACC', 'ATGGCGTTTTCAGGGTTCTGTTGGT', 'CAGGGTTCTGTTGGTGCACTAATCC', 'TTTCAGGGTTCTGTTGGTGCACTAA', 'TTTTCAGGGTTCTGTTGGTGCACTA', 'AACAACTTTTTCTATGGCGTTTTCA', 'TATGGCGTTTTCAGGGTTCTGTTGG', 'ACTTTTTCTATGGCGTTTTCAGGGT', 'GCCTTCCTCGCCGACGATGACCTCG', 'CCTCGCCGACGATGACCTCGGCGGG', 'CTATGGCGTTTTCAGGGTTCTGTTG', 'CGTTTTCAGGGTTCTGTTGGTGCAC', 'CTTTTTCTATGGCGTTTTCAGGGTT', 'AACTTTTTCTATGGCGTTTTCAGGG', 'GTTTTCAGG

In [10]:
kmerCounts_uu1 = dict(zip(kmer_uu1[0],kmer_uu1[1]))
kmerCounts_uu2 = dict(zip(kmer_uu2[0],kmer_uu2[1]))
kmerCounts_uu3 = dict(zip(kmer_uu3[0],kmer_uu3[1]))
kmerCounts_uu4 = dict(zip(kmer_uu4[0],kmer_uu4[1]))
kmerCounts_uu5 = dict(zip(kmer_uu5[0],kmer_uu5[1]))

kmerCounts_uniq_uu1 = dict((k, kmerCounts_uu1[k]) for k in unique_UU)
kmerCounts_uniq_uu2 = dict((k, kmerCounts_uu2[k]) for k in unique_UU)
kmerCounts_uniq_uu3 = dict((k, kmerCounts_uu3[k]) for k in unique_UU)
kmerCounts_uniq_uu4 = dict((k, kmerCounts_uu4[k]) for k in unique_UU)
kmerCounts_uniq_uu5 = dict((k, kmerCounts_uu5[k]) for k in unique_UU)

kmerCounts_uniq_UU = dict((k, kmerCounts_uniq_uu1[k]+kmerCounts_uniq_uu2[k]+kmerCounts_uniq_uu3[k]+kmerCounts_uniq_uu4[k]+kmerCounts_uniq_uu5[k]) for k in unique_UU)

kmerCounts_uniq_UU


{'ACACCGGCGCCGCCGCCTTCCTCGC': 37,
 'CCTTCCTCGCCGACGATGACCTCGG': 38,
 'TTCCTCGCCGACGATGACCTCGGCG': 38,
 'GGTTCTGTTGGTGCACTAATCCTGC': 27,
 'CGCCTTCCTCGCCGACGATGACCTC': 37,
 'CGCCGCCTTCCTCGCCGACGATGAC': 37,
 'AACACCGGCGCCGCCGCCTTCCTCG': 37,
 'GCGTTTTCAGGGTTCTGTTGGTGCA': 34,
 'TTTCTATGGCGTTTTCAGGGTTCTG': 34,
 'CGCCGCCGCCTTCCTCGCCGACGAT': 37,
 'TCAGGGTTCTGTTGGTGCACTAATC': 34,
 'GCCGCCGCCTTCCTCGCCGACGATG': 37,
 'TTTTTCTATGGCGTTTTCAGGGTTC': 34,
 'TGGCGTTTTCAGGGTTCTGTTGGTG': 34,
 'GGGTTCTGTTGGTGCACTAATCCTG': 34,
 'GCGCCGCCGCCTTCCTCGCCGACGA': 37,
 'TTCTATGGCGTTTTCAGGGTTCTGT': 34,
 'GCCGACGATGACCTCGGCGGGCTGT': 38,
 'CACCGGCGCCGCCGCCTTCCTCGCC': 37,
 'CAACTTTTTCTATGGCGTTTTCAGG': 34,
 'GCCGCCTTCCTCGCCGACGATGACC': 37,
 'ATGGCGTTTTCAGGGTTCTGTTGGT': 34,
 'CAGGGTTCTGTTGGTGCACTAATCC': 34,
 'TTTCAGGGTTCTGTTGGTGCACTAA': 34,
 'TTTTCAGGGTTCTGTTGGTGCACTA': 34,
 'AACAACTTTTTCTATGGCGTTTTCA': 34,
 'TATGGCGTTTTCAGGGTTCTGTTGG': 34,
 'ACTTTTTCTATGGCGTTTTCAGGGT': 34,
 'GCCTTCCTCGCCGACGATGACCTCG': 37,
 'CCTCGCCGACGA

In [11]:
def intersect(df0, df1):
    df0['gene'] = 'NA'
    for i in range(len(df0)):
        for j in range(len(df1)):
            if (df0.iloc[i,1] >= df1.iloc[j,1]) and (df0.iloc[i,2] <= df1.iloc[j,2]):
                df0.iloc[i,5] = df1.iloc[j,4]
    
    return df0


df_uu1 = intersect(df_uu1, gene_coord[gene_coord[0]=='B73V4_ctg150'])

In [12]:
df_uu1[df_uu1['seq'].str.contains('TTTTCTATGGCGTTTTCAGGGTTCT')]

,chr,start,end,seq,kmer,gene
469,B73V4_ctg150,598344,598395,CAACAACTTTTTCTATGGCGTTTTCAGGGTTCTGTTGGTGCACTAA...,"[CAACAACTTTTTCTATGGCGTTTTC, AACAACTTTTTCTATGGC...",Zm00001d000009
470,B73V4_ctg150,598344,598395,CAACAACTTTTTCTATGGCGTTTTCAGGGTTCTGTTGGTGCACTAA...,"[CAACAACTTTTTCTATGGCGTTTTC, AACAACTTTTTCTATGGC...",Zm00001d000009
471,B73V4_ctg150,598345,598396,AACAACTTTTTCTATGGCGTTTTCAGGGTTCTGTTGGTGCACTAAT...,"[AACAACTTTTTCTATGGCGTTTTCA, ACAACTTTTTCTATGGCG...",Zm00001d000009
472,B73V4_ctg150,598345,598396,AACAACTTTTTCTATGGCGTTTTCAGGGTTCTGTTGGTGCACTAAT...,"[AACAACTTTTTCTATGGCGTTTTCA, ACAACTTTTTCTATGGCG...",Zm00001d000009
473,B73V4_ctg150,598345,598396,AACAACTTTTTCTATGGCGTTTTCAGGGTTCTGTTGGTGCACTAAT...,"[AACAACTTTTTCTATGGCGTTTTCA, ACAACTTTTTCTATGGCG...",Zm00001d000009
474,B73V4_ctg150,598345,598396,AACAACTTTTTCTATGGCGTTTTCAGGGTTCTGTTGGTGCACTAAT...,"[AACAACTTTTTCTATGGCGTTTTCA, ACAACTTTTTCTATGGCG...",Zm00001d000009
475,B73V4_ctg150,598345,598396,AACAACTTTTTCTATGGCGTTTTCAGGGTTCTGTTGGTGCACTAAT...,"[AACAACTTTTTCTATGGCGTTTTCA, ACAACTTTTTCTATGGCG...",Zm00001d000009
476,B73V4_ctg150,598345,598396,AACAACTTTTTCTATGGCGTTTTCAGGGTTCTGTTGGTGCACTAAT...,"[AACAACTTTTTCTATGGCGTTTTCA, ACAACTTTTTCTATGGCG...",Zm00001d000009


### Each kmer is unique to a gene

In [13]:
geneCounts = {}
for kmer in kmerCounts_uniq_UU:
    t = df_uu1[df_uu1['seq'].str.contains(kmer)][['gene']].drop_duplicates().values
    try:
        assert len(t) == 1
    except Exception:
        print('multiple alignment')
        print(kmer)
    print(t[0][0])
    if t[0][0] not in geneCounts:
        geneCounts[t[0][0]] = kmerCounts_uniq_UU[kmer]
    else:
        geneCounts[t[0][0]] += kmerCounts_uniq_UU[kmer]
    

Zm00001d000001
Zm00001d000001
Zm00001d000001
Zm00001d000009
Zm00001d000001
Zm00001d000001
Zm00001d000001
Zm00001d000009
Zm00001d000009
Zm00001d000001
Zm00001d000009
Zm00001d000001
Zm00001d000009
Zm00001d000009
Zm00001d000009
Zm00001d000001
Zm00001d000009
Zm00001d000001
Zm00001d000001
Zm00001d000009
Zm00001d000001
Zm00001d000009
Zm00001d000009
Zm00001d000009
Zm00001d000009
Zm00001d000009
Zm00001d000009
Zm00001d000009
Zm00001d000001
Zm00001d000001
Zm00001d000009
Zm00001d000009
Zm00001d000009
Zm00001d000009
Zm00001d000009
Zm00001d000009
Zm00001d000009
Zm00001d000001
Zm00001d000001
Zm00001d000009
Zm00001d000001
Zm00001d000001
Zm00001d000009
Zm00001d000001
Zm00001d000001
Zm00001d000001
Zm00001d000001
Zm00001d000009
Zm00001d000001
Zm00001d000001
Zm00001d000001
Zm00001d000001
Zm00001d000009


### 971 k-mers unique to Ufo1-1 that are mapped to Zm00001d000001(polyphenol oxidase1)
### 911 k-mers unique to Ufo1-1 that are mapped to Zm00001d000009(our candidate gene)

### Further analysis revealed that unique k-mers in Zm00001d000001 resulted from SNP variations whereas Zm00001d000009 was due to structural variation (transposable element insertion)

In [14]:
geneCounts

{'Zm00001d000001': 971, 'Zm00001d000009': 911}